In [ ]:
using FFTW, DelimitedFiles

# 入力パラメータの設定
folder_path = raw"D:\新しいフォルダー (2)"  # 処理するフォルダのパスを指定
output_folder = raw"D:\新しいフォルダー (3)"  # 出力フォルダのパスを指定
interval_sec = 1/60  # テキストデータ間隔（秒）
lowpass_threshold = 5  # ローパスフィルタの閾値（Hz）

# 指定フォルダ内のテキストファイルを取得
files = filter(x -> endswith(x, ".txt"), readdir(folder_path, join=true))
sort!(files)

# データの格納用辞書
velocity_data = Dict()
coordinates = Set{Tuple{Float64, Float64}}()

timesteps = length(files)  # タイムステップ数

# 各ファイルを読み込み、速度データを格納
for file in files
    data = readdlm(file)
    for row in eachrow(data)
        x, y, vx, vy, speed = row[1:5]
        push!(coordinates, (x, y))  # 修正: 直接push!を使用
        if !haskey(velocity_data, (x, y))
            velocity_data[(x, y)] = Float64[]
        end
        push!(velocity_data[(x, y)], speed)
    end
end

# FFT、ローパスフィルタ、逆FFTを適用
t_filtered = Dict()
fs = 1 / interval_sec  # サンプリング周波数
freqs = fftfreq(timesteps, 1/fs)

for coord in coordinates
    vels = velocity_data[coord]
    fft_data = fft(vels)
    fft_data[abs.(freqs) .> lowpass_threshold] .= 0  # ローパスフィルタ適用
    t_filtered[coord] = real(ifft(fft_data))
end

# フィルタ後のデータを新しいテキストファイルに出力
for (i, file) in enumerate(files)
    data = readdlm(file)
    new_data = copy(data)
    for row in eachrow(new_data)
        x, y = row[1:2]
        row[5] = t_filtered[(x, y)][i]  # 速度値を更新
    end
    output_file = joinpath(output_folder, basename(file))
    writedlm(output_file, new_data)
end

# 座標 (8,8) の処理結果をログ出力
if haskey(t_filtered, (8.0, 8.0))
    println("Original velocities at (8,8):", velocity_data[(8.0, 8.0)])
    println("Filtered velocities at (8,8):", t_filtered[(8.0, 8.0)])
end

Original Vx at (8,8):[0.0, 0.25599787, 0.12304719, 0.0, 0.033410154, 0.16944233, -0.33411599, 0.0, 0.061736457, 0.12866239]
Filtered Vx at (8,8):[1.1102230246251566e-17, 0.25599787, 0.12304719000000004, -2.402130697178538e-17, 0.03341015399999996, 0.16944233, -0.33411599000000003, -5.308065101698713e-17, 0.061736457000000036, 0.12866239]
Original Vy at (8,8):[0.0, 4.7629196, 4.8621675, 0.0, 5.9788952, 0.14707457, 4.8679387, 0.0, 6.5947764, 0.1576778]
Filtered Vy at (8,8):[0.0, 4.762919599999999, 4.862167500000001, -3.424752088599552e-16, 5.9788952, 0.1470745700000002, 4.8679387, -4.136568516279746e-16, 6.594776400000001, 0.1576778000000001]
Original Speed at (8,8):[0.0, 4.7697943, 4.8637243, 0.0, 5.9789885, 0.22436941, 4.8793914, 0.0, 6.5950654, 0.20350995]
Filtered Speed at (8,8):[0.0, 4.7697943, 4.8637243, 1.625070563578694e-16, 5.9789885, 0.2243694100000001, 4.8793914, -6.025656186061451e-17, 6.5950654, 0.20350995000000083]
